In [ ]:
#A temporary atco generator. This is not part of the problem.
#As the atco employee list is expected to be given.
import random

positions = [2, 5, 12, 8, 3]
min_required = [(x//2+(x-x//2)*2)*5 for x in positions]

def generate_atco(min_required=min_required):
    ratings = ['A', 'B', 'C', 'D', 'E', 'F']
    atcos=[]
    c,r=1,0
    #adding users with only 1 qualification
    for i in min_required:
        for j in range(i-random.randint(0,5)):
            atco_id = f'ATCO_{c}'
            c+=1
            atcos.append({'id': atco_id, 'ratings': ratings[r]})
        r+=1
    #adding users with more than 1 qualifiction
    for i in range(40):
        atco_id = f'ATCO_{c}'
        c+=1
        num_ratings = random.randint(2, 3)  # Each user can have 2 to 3 ratings
        atco_ratings = random.sample(ratings, num_ratings)
        atcos.append({'id': atco_id, 'ratings': atco_ratings})
    return atcos

atcos=generate_atco(min_required)


In [128]:
# A temporary atco generator. This is not part of the problem.
# As the atco employee list is expected to be given.

import random

def generate_random_atcos(n):
    atcos = {}
    ratings_list = ['A', 'B', 'C', 'D', 'E', 'F']
    
    for i in range(1, n+1):
        num_ratings = random.randint(1, 3)  # Each atco has between 1 and 3 ratings
        atco_ratings = random.sample(ratings_list, num_ratings)
        preferences = {}
        for rating in atco_ratings:
            preferences[rating] = random.randint(1, 30)  # Preference scores between 1 and 30 - this is essentially credit
        atcos[f'Atco{i}'] = {
            'ratings': atco_ratings,
            'preferences': preferences
        }
    return atcos

def add_admin_atco(c, atcos):
    ratings_list = ['A', 'B', 'C', 'D', 'E', 'F']
    atcos[f'Admin_Atco{c}'] = {
            'ratings': ['A', 'B', 'C', 'D', 'E', 'F'],
            'preferences': {rating: 10 for rating in ratings_list}
        }


In [133]:
import pulp

# Define the ratings and the required number of positions per rating
ratings = {
    'A': 2,
    'B': 5,
    'C': 12,
    'D': 8,
    'E': 3,
    'F': 4,
}

# Calculate total atcos and required number of atcos per position
n = 0
for r in ratings:
    x = ratings[r]
    ratings[r] = (x // 2 + (x - x // 2) * 2) * 5
    n += ratings[r]

# Generate atcos
atcos = generate_random_atcos(n)

# Create a list of all assignments (atco, rating) they are qualified for
# and a dictionary of their preference scores
assignments = []
preferences = {}
for atco, data in atcos.items():
    quals = data['ratings']
    prefs = data['preferences']
    for rating in quals:
        assignments.append((atco, rating))
        preferences[(atco, rating)] = prefs.get(rating, 0)  # Default to 0 if not specified

# Create the problem variable to contain the problem data
prob = pulp.LpProblem("Atco_Rating_Assignment_With_Preferences", pulp.LpMaximize)

# Decision variables: x[(atco, rating)] = 1 if atco is assigned to rating, 0 otherwise
x = pulp.LpVariable.dicts("assign", assignments, cat='Binary')

# Objective Function: Maximize total preference score
prob += pulp.lpSum([preferences[(atco, rating)] * x[(atco, rating)] for (atco, rating) in assignments]), "TotalPreferenceScore"

# Constraints

# 1. Each rating must be assigned the required number of atcos
for rating in ratings:
    prob += (
        pulp.lpSum([x[(atco, rating)] for atco in atcos if (atco, rating) in assignments])
        == ratings[rating],
        f"RatingRequirement_{rating}"
    )

# 2. Each atco must be assigned to exactly one rating
for atco in atcos:
    prob += (
        pulp.lpSum([x[(atco, rating)] for rating in atcos[atco]['ratings']])
        == 1,
        f"AtcoAssignment_{atco}"
    )

# Solve the problem
prob.solve()

# Output the results
print("Status:", pulp.LpStatus[prob.status])
if prob.status == 1:
    print("\nAssignment Results:")
    total_pref_score = 0
    for (atco, rating) in assignments:
        if x[(atco, rating)].varValue == 1:
            pref_score = preferences[(atco, rating)]
            total_pref_score += pref_score
            print(f"{atco} is assigned to rating {rating} with preference score {pref_score}")
    print(f"\nTotal Preference Score: {total_pref_score}")
else:
    print("No feasible solution found.")



Status: Optimal

Assignment Results:
Atco1 is assigned to rating D with preference score 28
Atco2 is assigned to rating C with preference score 17
Atco3 is assigned to rating F with preference score 5
Atco4 is assigned to rating D with preference score 1
Atco5 is assigned to rating B with preference score 10
Atco6 is assigned to rating A with preference score 16
Atco7 is assigned to rating D with preference score 25
Atco8 is assigned to rating F with preference score 1
Atco9 is assigned to rating A with preference score 6
Atco10 is assigned to rating C with preference score 8
Atco11 is assigned to rating C with preference score 8
Atco12 is assigned to rating C with preference score 7
Atco13 is assigned to rating D with preference score 3
Atco14 is assigned to rating C with preference score 21
Atco15 is assigned to rating C with preference score 1
Atco16 is assigned to rating E with preference score 15
Atco17 is assigned to rating F with preference score 23
Atco18 is assigned to rating 

In [127]:
import pulp
import random

# A temporary atco generator. This is not part of the problem.
# As the atco employee list is expected to be given.
def generate_random_atcos(n):
    atcos = {}
    ratings_list = ['A', 'B', 'C', 'D', 'E', 'F']
    
    for i in range(1, n+1):
        num_ratings = random.randint(1, 3)  # Each atco has between 1 and 3 ratings
        atco_ratings = random.sample(ratings_list, num_ratings)
        preferences = {}
        for rating in atco_ratings:
            preferences[rating] = random.randint(1, 20)  # Preference scores between 1 and 20
        atcos[f'Atco{i}'] = {
            'ratings': atco_ratings,
            'preferences': preferences
        }
    return atcos

def add_admin_atco(atcos, admin_atco_count):
    ratings_list = ['A', 'B', 'C', 'D', 'E', 'F']
    admin_atco_name = f'admin_atco_{admin_atco_count}'
    preferences = {rating: 10 for rating in ratings_list}  # Preference score of 10 for all ratings
    atcos[admin_atco_name] = {
        'ratings': ratings_list,
        'preferences': preferences
    }
    return atcos

# main function where the magic happens
def assign_atcos_to_ratings(ratings, atcos):
    admin_atco_count = 0  # Counter for admin atcos

    while True:
        # Create a list of all assignments and a dictionary of preferences
        assignments = []
        preferences = {}
        for atco, data in atcos.items():
            quals = data['ratings']
            prefs = data['preferences']
            for rating in quals:
                assignments.append((atco, rating))
                preferences[(atco, rating)] = prefs.get(rating, 0)  # Default to 0 if not specified

        # Create the problem variable to contain the problem data
        prob = pulp.LpProblem("Atco_Rating_Assignment_With_Preferences", pulp.LpMaximize)

        # Decision variables: x[(atco, rating)] = 1 if atco is assigned to rating, 0 otherwise
        x = pulp.LpVariable.dicts("assign", assignments, cat='Binary')

        # Objective Function: Maximize total preference score
        prob += pulp.lpSum([preferences[(atco, rating)] * x[(atco, rating)] for (atco, rating) in assignments]), "TotalPreferenceScore"

        # Constraints

        # 1. Each rating must be assigned the required number of atcos
        for rating in ratings:
            prob += (
                pulp.lpSum([x[(atco, rating)] for atco in atcos if (atco, rating) in assignments])
                == ratings[rating],
                f"RatingRequirement_{rating}"
            )

        # 2. Each atco must be assigned to exactly one rating
        for atco in atcos:
            prob += (
                pulp.lpSum([x[(atco, rating)] for rating in atcos[atco]['ratings']])
                <= 1,
                f"AtcoAssignment_{atco}"
            )

        # Solve the problem
        prob.solve()

        # Check if a feasible solution was found
        if prob.status == 1:
            # Feasible solution found
            break
        else:
            # Infeasible solution; add an admin atco and try again
            admin_atco_count += 1
            atcos = add_admin_atco(atcos, admin_atco_count)

    # Output the results
    print("Status:", pulp.LpStatus[prob.status])
    if prob.status == 1:
        print("\nAssigned Atcos and Their Ratings:\n")
        total_pref_score = 0
        assigned_atcos = {}  # Dictionary to store assigned atcos per rating
        for atco, data in atcos.items():
            assigned = False
            for rating in data['ratings']:
                if x.get((atco, rating)) and x[(atco, rating)].varValue == 1:
                    pref_score = preferences[(atco, rating)]
                    total_pref_score += pref_score
                    if rating not in assigned_atcos:
                        assigned_atcos[rating] = []
                    assigned_atcos[rating].append(atco)
                    assigned = True
                    break  # Since the atco is assigned to one rating
            if not assigned:
                print(f"{atco}: Not assigned to any rating.")

        # Grouping atcos assigned to each rating into 5 groups
        print("\nGrouping of Atcos Assigned to Each Rating:\n")
        for rating, atcos_list in assigned_atcos.items():
            print(f"Rating {rating}:")
            groups = split_list(atcos_list, 5)
            for i, group in enumerate(groups, 1):
                print(f"  Group {i}: {group}")
            print()

        print(f"Total Preference Score: {total_pref_score}")
        if admin_atco_count > 0:
            print(f"\nNumber of admin atcos added: {admin_atco_count}")
    else:
        print("No feasible solution found.")

# split the list for each grp into number of day-cycles
def split_list(lst, n):
    if len(lst) < n:
        return [lst[i:i+1] for i in range(len(lst))]
    else:
        k, m = divmod(len(lst), n)
        return [lst[i*(k+1) : i*(k+1)+(k+1)] if i < m else lst[i*k + m : (i+1)*k + m] for i in range(n)]

# Ratings and positions per rating
ratings = {
    'A': 2,
    'B': 5,
    'C': 12,
    'D': 8,
    'E': 3,
    'F': 4,
}
no_of_atcos = copy.deepcopy(ratings)

# Calculate total atcos and initial required number of atcos per position
n = 0
for r in ratings:
    x = ratings[r]
    no_of_atcos[r] = (x // 2 + (x - x // 2) * 2) * 5
    n += no_of_atcos[r]
atcos = generate_random_atcos(n)
assign_atcos_to_ratings(no_of_atcos, atcos)


Status: Optimal

Assigned Atcos and Their Ratings:


Grouping of Atcos Assigned to Each Rating:

Rating F:
  Group 1: ['Atco1', 'Atco11', 'Atco35', 'Atco36', 'Atco45', 'Atco47']
  Group 2: ['Atco49', 'Atco69', 'Atco87', 'Atco88', 'Atco102', 'Atco105']
  Group 3: ['Atco113', 'Atco131', 'Atco146', 'Atco166', 'Atco170', 'Atco174']
  Group 4: ['Atco178', 'Atco181', 'Atco182', 'Atco191', 'Atco199', 'Atco212']
  Group 5: ['Atco228', 'Atco230', 'Atco231', 'Atco241', 'Atco243', 'Atco248']

Rating D:
  Group 1: ['Atco2', 'Atco7', 'Atco12', 'Atco14', 'Atco18', 'Atco19', 'Atco33', 'Atco37', 'Atco39', 'Atco40', 'Atco46', 'Atco54']
  Group 2: ['Atco62', 'Atco64', 'Atco67', 'Atco68', 'Atco70', 'Atco73', 'Atco77', 'Atco78', 'Atco81', 'Atco83', 'Atco89', 'Atco93']
  Group 3: ['Atco95', 'Atco98', 'Atco101', 'Atco109', 'Atco112', 'Atco117', 'Atco119', 'Atco122', 'Atco124', 'Atco128', 'Atco129', 'Atco135']
  Group 4: ['Atco137', 'Atco138', 'Atco145', 'Atco150', 'Atco156', 'Atco157', 'Atco163', 'Atco164',